# Sparkland Optimisation Project

Sparkland is a fictional energy utilities provider. Sparkland has a number of issues they seek to address through progressively optimising the grid, using LP, MILP and Dynamic techniques. They seek the help of an optimisation expert to perform these optimisations.

Although Sparkland is a fictional company, they certainly behave like a real client; setting optimisation parameters, assessing your answer, and then changing the goal posts because they have new circumstances, or they simply forgot to provide key information! Certainly great practice for consultants in waiting. 

This actually happens to be a great reaal-world example: LP and MILP is a vital cog that underpins the electricity market on the Australian eastern seaboard so this kind of project has real world application (although real-life data would be a lot messier)

Lets open our first email!

***

`From: Sparkland Admin
 Date: 23/03/2021
 RE: Generator Optimisation`

_Sparkland supplies electricity to a small region from four generators running on natural gas. Over the years we have built a network of transmission lines, running from our generators to substation nodes around the region._

_We have provided you with the following data files:_
*  _nodes.csv gives the location of each node (units are kilometres) and the current demand (MW) at that node_
*  _grid.csv gives all the connections between the nodes that make up our grid_

_Some nodes are classified as generator nodes. Due to various factors, these generators have different capacities and costs for producing electricity, as shown in the following table:_


| Node | Capacity (MW) | Cost ($/MW)|
| :-: | :-: | :-: |
| 20 | 406 | 65 | 
| 45 | 838 | 70 |
| 35 | 818 | 74 | 
| 37 | 625 | 82 |

_Please provide us with the optimal cost for meeting the current demand over a whole day from our generators._

_Regards,_

_Sparkland_

***

This is a pretty standard request and has no network related constraints. This could be easily solved with Excel or a pen and paper, however this is a good opportunity to set up a linear programming model and see how it works. So lets set it up in our Gurobi package and analyse it.

Firstly we will import our favourite packages (this assumes that you have gone to the Gurobi website, downloaded the latest version, and obtained the appropriate licence __[https://www.gurobi.com/academia/academic-program-and-licenses/](https://www.gurobi.com/academia/academic-program-and-licenses/)__), and we'll read in the data using pandas. We'll use math later when we need to calculate some distances.

In [1]:
from gurobipy import *
import math
import pandas as pd

nodes = pd.read_csv('nodes.csv')
arcs = pd.read_csv('grid.csv')

nodes.head()

,Node,X,Y,Demand
0,0,90,5,55
1,1,148,132,29
2,2,173,33,53
3,3,59,130,57
4,4,52,9,88


In [2]:
arcs.head()

,Arc,Node1,Node2
0,0,8,27
1,1,27,8
2,2,8,10
3,3,10,8
4,4,24,25


So the nodes.csv lists the nodes, the x/y map coordinates of the node, and the total demand of the node. The arc.csv lists the arcs connecting the nodes, giving the start `['Node1']` and end node `['Node2']`. Note that each arc is listed twice, as each node is listed as a start _and_ end node.

Now we should first check what the demand profile looks like at the generator node, as this may colour our analysis.


In [3]:
G = [20, 45, 35, 37]
nodes['Demand'][G]

20    0
45    0
35    0
37    0
Name: Demand, dtype: int64

No demand at the generator node. It makes intuitive sense, but better to make sure. 

Okay so now we want to start writing our problem as an LP problem. It is useful to think of problems containing the following elements:
*  Sets: In this problem there are a few sets. The set of all arcs, the set of all nodes, the  set of generator nodes (which is a subset of all nodes). Sets and subsets allow some flexibility in how things are structured. 
*  Data: This is the data relevant to the problem. This data can be a global constant, or be specific to different elements of a set. In this case, we have demand data for each node, capacity for generator nodes, etc
*  Decision Variable: this is the variable that we are trying to target. This can sometimes be a bit tricky. In this case, we will be looking at minimising costs, but the decision variable will actually be energy transmitted. We will be looking to minimise the amount of energy produced at expensive generators, thereby reducing the overall cost. 
*  Objective: This is the mathematical expression that minimises/maximises what we are looking for. 
*  Constraints: mathematical constraints on the optimisations. More on this later

So to start with, we will define the set of nodes and arcs as sets so we can play with them.  

In [4]:
#sets
A = arcs['Arc']
N = nodes['Node']

Next, we can load in our generator data

In [5]:
# Email 1 data 
costs= { 20: 65, 45: 70, 35: 74, 37: 82}
supply = { 20: 406, 45: 838, 35: 818, 37: 654}

The email request is asking us to meet total demand at the cheapest possible price. There is no indication that we need to use any particular transmission arcs, so this is a simple total demand problem.

In [6]:
total_demand = nodes['Demand'].sum()
total_demand

2029

Okay, lets start setting up our model

In [7]:
m = Model('Sparkland')

Academic license - for non-commercial use only - expires 2022-03-31
Using license file C:\Users\kateb\gurobi.lic


Now we need to select a decision variable. We are trying to minimise the cost of energy so 'energy provided' seems like a pretty good decision variable to me. We will be trying to reduce costs node-wise. 

_Gurobi has different ways of declaring variables, so check to see how you like to do it._

In [8]:
# Email 1 variables

X = {n: m.addVar() for n in N}
X

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>,
 7: <gurobi.Var *Awaiting Model Update*>,
 8: <gurobi.Var *Awaiting Model Update*>,
 9: <gurobi.Var *Awaiting Model Update*>,
 10: <gurobi.Var *Awaiting Model Update*>,
 11: <gurobi.Var *Awaiting Model Update*>,
 12: <gurobi.Var *Awaiting Model Update*>,
 13: <gurobi.Var *Awaiting Model Update*>,
 14: <gurobi.Var *Awaiting Model Update*>,
 15: <gurobi.Var *Awaiting Model Update*>,
 16: <gurobi.Var *Awaiting Model Update*>,
 17: <gurobi.Var *Awaiting Model Update*>,
 18: <gurobi.Var *Awaiting Model Update*>,
 19: <gurobi.Var *Awaiting Model Update*>,
 20: <gurobi.Var *Awaiting Model Update*>,
 21: <gurobi.Var *Awaiting Model Update*>,
 22: <gurobi.Var *Awaiting Model Update*>,
 23: <gurobi.Var *Awa

So what have we done here? Gurobi has created a special gurobi variable for each node in the set N, and we have labeled them n. We haven't run the model yet so there is nothing in the variable yet. 

Now we are going to set the objective. 

In [9]:
# Email 1 objective

m.setObjective(24*(quicksum(costs[n]*X[n] for n in costs)),GRB.MINIMIZE)

Let's break this down a little. The 24 is due to the format of the data. We are seeking to minimise a daily cost, but the generator data is given in MW/h, so multiply by 24 to get the daily figure. The 'for n in costs' statement restricts the calculation to the nodes that are represented in the set 'costs'. The X is the amount of energy provided by generator 'n' and that is multiplied by the cost of n. The GRB.MINIMIZE tells gurobi which direction to optimise in.

The quicksum function is a special Gurobi function. It is essentially just a sum function (similar to sigma notation) however using some other sum function (eg numpy) would not work as well. 

Finally, we need to state our constraints. We have two constraints in this case:

In [10]:
# Email 1 constraints

# The amount of energy provided by generators meets the total demand of the grid.
m.addConstr(quicksum(X[n] for n in costs) == total_demand)

# each generator cannot exceed its supply capacity
for n in costs:
    m.addConstr(X[n] <= supply[n])

that's it, let's see how it performs when we run it

In [11]:
m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 50 columns and 8 nonzeros
Model fingerprint: 0x03106582
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 2e+03]
Presolve removed 5 rows and 50 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4353600e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  3.435360000e+06
Minimum cost = $ 3435360.0


So the optimal cost is \$3,434,360. We've emailed the results to our client and they accept the results. This is a pretty simplistic example, and it would be pretty easy to double check our answer by just summing the total demand in excel and meeting the demand by maxing out the cheapest generators first. We can see that this is exactly what happened by inspecting the Gurobi model variables, observe that the most expensive generator (n = 37) generates zero energy.

In [12]:
X

{0: <gurobi.Var C0 (value 0.0)>,
 1: <gurobi.Var C1 (value 0.0)>,
 2: <gurobi.Var C2 (value 0.0)>,
 3: <gurobi.Var C3 (value 0.0)>,
 4: <gurobi.Var C4 (value 0.0)>,
 5: <gurobi.Var C5 (value 0.0)>,
 6: <gurobi.Var C6 (value 0.0)>,
 7: <gurobi.Var C7 (value 0.0)>,
 8: <gurobi.Var C8 (value 0.0)>,
 9: <gurobi.Var C9 (value 0.0)>,
 10: <gurobi.Var C10 (value 0.0)>,
 11: <gurobi.Var C11 (value 0.0)>,
 12: <gurobi.Var C12 (value 0.0)>,
 13: <gurobi.Var C13 (value 0.0)>,
 14: <gurobi.Var C14 (value 0.0)>,
 15: <gurobi.Var C15 (value 0.0)>,
 16: <gurobi.Var C16 (value 0.0)>,
 17: <gurobi.Var C17 (value 0.0)>,
 18: <gurobi.Var C18 (value 0.0)>,
 19: <gurobi.Var C19 (value 0.0)>,
 20: <gurobi.Var C20 (value 406.0)>,
 21: <gurobi.Var C21 (value 0.0)>,
 22: <gurobi.Var C22 (value 0.0)>,
 23: <gurobi.Var C23 (value 0.0)>,
 24: <gurobi.Var C24 (value 0.0)>,
 25: <gurobi.Var C25 (value 0.0)>,
 26: <gurobi.Var C26 (value 0.0)>,
 27: <gurobi.Var C27 (value 0.0)>,
 28: <gurobi.Var C28 (value 0.0)>,
 29

Anyway a few days later, Sparkland send a new email 

***

_Thank you for your initial estimate. However, we did not mention that our transmission lines actually lose electricity along them. This loss can be estimated as 0.1\% per km._

_Could you revise your proposal to take this into account? Please provide us with the optimal cost for meeting the current demand over a whole day from our generators._

***

So now we will need to make use of the set of arcs and make sure that we use the most efficient pathways. It is no use using a cheap generator if the transmission costs are too expensive. At the very least we will need to generate more energy to cover the losses. 

For this new data, we need to build in the transmission loss and calculate the lengths of all the arcs. 

In [13]:
# Email 2 data

loss = 0.001 # Loss factor (% per km)

# Calculate lengths of each arc (euclidean distance)
distance = [math.hypot(
    nodes['X'][arcs['Node1'][a]]-nodes['X'][arcs['Node2'][a]],
    nodes['Y'][arcs['Node1'][a]]-nodes['Y'][arcs['Node2'][a]]) for a in A]

Time to start a new model. We also need to review our decision variables. The recent communication suggests we need to govern the power flow on any arc, so we will need an additional decision variable to account for arc power flow. 

(The model 'm' is still active, but we need to declare our previous X variable again to clear the current contents)

In [14]:
# Email 2 variables

# X gives amount generated by generators at node n 
X = {n: m.addVar() for n in N}

# Y gives flow on arc a 
Y = {a: m.addVar() for a in A}


Our objective does not change, but the constraints start to look more complicated

In [15]:
# email 2 objective

m.setObjective(24*(quicksum(costs[n]*X[n] for n in costs)),GRB.MINIMIZE)

In [16]:
# Email 2 constraints

for n in N:
    # balancing constraint
    m.addConstr(quicksum(Y[a]*(1-loss*distance[a]) for a in A if arcs['Node2'][a] == n) + X[n]  ==
                quicksum(Y[a] for a in A if arcs['Node1'][a] == n) + nodes['Demand'][n])
    if n in supply:
        # generator must generate within given capacity limits
        m.addConstr(X[n] <= supply[n])
    else:
        # forcing constraint
        m.addConstr(X[n] == 0)

The balancing constraint looks a bit confusing, and this is can actually be written as 4 different constraints (arguably that might be even more complicated!). The LHS of the equation relates to power entering a node n. any power entering a node will be curtailed by the loss factor, and if that node is a generator node, then additional power Y will be added. The RHS of the equation relates to the power leaving the node, and also the demand for that node. So essentially, the curtailed power entering the node must be equal to the power leaving the node plus the demand of that node (with some special rules for generators)

The forcing contraint is a bit more straightforward but also has some nuance. Power created at generator nodes cannot exceed the supply limits which makes sense, however this constraint on its own would not work. You must specify that non-generator nodes will have a zero output otherwise the model will simply assign Y values to nodes and bring the objective value all the way to zero.

Okay lets test our model

In [17]:
m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 105 rows, 294 columns and 496 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.029000e+03   0.000000e+00      0s
     189    3.6692435e+06   0.000000e+00   0.000000e+00      0s

Solved in 189 iterations and 0.02 seconds
Optimal objective  3.669243450e+06
Minimum cost = $ 3669243.450357632


$3,669,243 is a bit more expensive than the last model, and that makes sense. We have added loss factors which means the generators have to work harder to get the power out there. Lets see how the generation pattern looks this time.

Rather than just calling the whole variable and inspecting it (which gets pretty tedious for big variables), we can make use of some Gurobi tricks. For example, you can call the 'x' of a decision variable which will return the current value of that decision variable, and use regular python/pandas commands to sift through them. 

In [18]:
for n in N:
    if X[n].x >0:
        print('node', n, ': ', X[n].x)

node 20 :  406.0
node 35 :  818.0
node 37 :  89.06272884026016
node 45 :  838.0


This time the network needed to rely on the more expensive generator as the three cheaper generators did not have enough capacity to account for the transmission losses. 

Well Sparkland are very happy with this, but sure enough they come back with another email. 

***

_We have realised that your proposal will exceed the limits on some of our transmission lines. The following 18 lines can effectively handle any load:_

_30 31 32 33 36 37 48 49 88 89 92 93 96 97 118 119 122 123_

_However, all of the other lines have a limit of 100 MW. Could you revise your proposal to take this into account? Please provide us with the optimal cost for meeting the current demand over a whole day from our generators._

***

Looks like we have new data to incorporate into our model. This is actually pretty straightforward to set up. We firstly load the new data.

In [19]:
# Email 3 data

lowlimit = 100 # Transmission maximum limit (MW)
highs = [30,31,32,33,36,37,48,49,88,89,92,93,96,97,118,119,122,123] # Transmission arcs with no limit

The existing information (sets, data, objective, constraints) will all remain

In [20]:
# Email 3 variables

# X gives amount generated by generators at node n 
X = {n: m.addVar() for n in N}

# Y gives flow on arc a 
Y = {a: m.addVar() for a in A}

In [21]:
# Email 3 objective

m.setObjective(24*(quicksum(costs[n]*X[n] for n in costs)),GRB.MINIMIZE)

We need one new constraint that tells the model to curtail transmission capacity for all arcs EXCEPT for those on our list of high-capacity arcs. 

In [22]:
for a in A:
    # constrain maximum flow on arc a 
    if not a in highs:
        m.addConstr(Y[a] <= lowlimit)

for n in N:
    # Balancing the power flow at each node 
    m.addConstr(quicksum(Y[a]*(1-loss*distance[a]) for a in A if arcs['Node2'][a] == n) + X[n]  ==
                quicksum(Y[a] for a in A if arcs['Node1'][a] == n) + nodes['Demand'][n])
    if n in supply:
        # generator must generate within given capacity limits
        m.addConstr(X[n] <= supply[n])
    else:
        # forcing constraint
        m.addConstr(X[n] == 0)

Let's hit go

In [23]:
m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 381 rows, 538 columns and 1160 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.029000e+03   0.000000e+00      0s
     208    3.6804847e+06   0.000000e+00   0.000000e+00      0s

Solved in 208 iterations and 0.02 seconds
Optimal objective  3.680484670e+06
Minimum cost = $ 3680484.669555652


Lets see how our variables look now. We'll round the values as those floating decimals are irritating to read.

In [24]:
for n in N:
    if X[n].x >0:
        print('node', n, ': ', round(X[n].x,2))

node 20 :  400.0
node 35 :  818.0
node 37 :  99.53
node 45 :  838.0


In [25]:
for a in A:
    if Y[a].x >0:
        print('arc', a ,': ', round(Y[a].x,2))

arc 1 :  46.38
arc 2 :  76.21
arc 5 :  28.31
arc 6 :  21.31
arc 8 :  44.56
arc 11 :  44.0
arc 17 :  32.96
arc 23 :  90.88
arc 24 :  34.01
arc 27 :  46.54
arc 28 :  31.36
arc 31 :  405.93
arc 32 :  341.75
arc 37 :  261.86
arc 45 :  53.43
arc 47 :  54.64
arc 48 :  114.77
arc 53 :  40.9
arc 61 :  24.66
arc 65 :  100.0
arc 68 :  34.7
arc 70 :  0.15
arc 80 :  29.95
arc 82 :  100.0
arc 85 :  100.0
arc 89 :  260.33
arc 93 :  168.55
arc 95 :  100.0
arc 97 :  220.47
arc 101 :  97.77
arc 104 :  63.19
arc 107 :  42.65
arc 110 :  96.02
arc 112 :  100.0
arc 114 :  74.06
arc 118 :  152.8
arc 123 :  297.29
arc 127 :  34.01
arc 130 :  55.93
arc 135 :  85.3
arc 139 :  52.68
arc 140 :  0.38
arc 147 :  79.87
arc 149 :  10.1
arc 155 :  28.34
arc 156 :  63.8
arc 160 :  27.16
arc 162 :  2.42
arc 164 :  44.9
arc 169 :  25.67
arc 173 :  3.94
arc 177 :  100.0
arc 178 :  100.0
arc 181 :  100.0
arc 183 :  75.15
arc 187 :  58.7
arc 188 :  100.0


In variable X, we are getting a really great picture of how energy flows throughout the system to meet demand. You can see that some arcs are maxed out to 100, some 'high' arcs are well beyond 100, while other arcs are not used at all. Model is working as planned.

Okay so setting that one up wasn't too bad at all! Getting the hang of this now! Can't wait for the next email!

***

_Thank you for helping satisfy this current demand. However, in practice demand changes over the day, and we are concerned about how our network will cope with peak demand times. We have broken the day into six time periods: Midnight to 4am, 4am to 8am, 8am to 12pm, 12pm to 4pm, 4pm to 8pm, and 8pm to midnight._

_Please see the attached data file:_

_nodes2.csv gives an update of our earlier data with demands for each node (MW) at each of these time periods
Could you revise your proposal to incorporate these changing demands? Please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our generators._

***

Hmm changing demand profile ey? I wonder what that looks like

In [26]:
nodes = pd.read_csv('nodes2.csv')

nodes.head()

,Node,X,Y,D0,D1,D2,D3,D4,D5
0,0,90,5,26,44,49,51,61,56
1,1,148,132,13,23,26,32,35,32
2,2,173,33,37,45,49,50,59,56
3,3,59,130,24,37,59,45,72,54
4,4,52,9,47,58,76,106,98,67


Things are getting a bit more complicated. Looks like we'll need to ensure that we optimise each time period and then stitch them together at the end. Mercifully, all of the other data sets stay the same, but we need to start fiddling with our objective, variables, and constraints. 

First thing we will do is turn the demand data into something that is easy to work with.

In [27]:
# Email 4 data 

T = range(6) # Set of daily time periods
D = [[nodes['D'+str(t)][n] for t in T] for n in N] # table of demands D[n][t] for each node

So now we have a 2D table that represents our demands for each node for each time period. Our decision variables will also now be dependent on the time of day, so in our new model we will adjust our variables and objective to reflect that. 


In [28]:
# Email 4 variables

# X gives amount generated by generators at node n in time period t 
X = {(n,t): m.addVar() for n in N for t in T}

# Y gives flow on arc a in time period t
Y = {(a,t): m.addVar() for a in A for t in T}

In [29]:
# Email 4 objective

m.setObjective(4*(quicksum(costs[n]*X[n,t] for n in costs for t in T)), GRB.MINIMIZE)

Here we can see our objective has changed slightly. Our generator decision variable now iterates over the six different periods of the day, so we actually have 6 hours of the day already calculated. So we need to multiply by four to get the full 24 hours. 

Our constraints will change slightly to accommodate the new variables. Substantively they do not change very much, we only need to make sure that we now iterate over all time periods for each arc/node/generator

In [30]:
# Email 4 constraints

for t in T:
    for a in A:
        if not a in highs:
            m.addConstr(Y[a,t] <= lowlimit)
            
    for n in N:
        # Balancing the power flow at each node and in each time period
        m.addConstr(quicksum(Y[a,t]*(1-loss*distance[a]) for a in A if arcs['Node2'][a] == n) + X[n,t]  ==
                    quicksum(Y[a,t] for a in A if arcs['Node1'][a] == n) + D[n][t])
        
        if n in supply:
            # Power generated by existing generators under capacity
            m.addConstr(X[n,t] <= supply[n])
        else: 
            # Forcing constraint
            m.addConstr(X[n,t] == 0)

In [31]:
m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2037 rows, 2002 columns and 5144 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.068300e+04   0.000000e+00      0s
    1377    3.2215060e+06   0.000000e+00   0.000000e+00      0s

Solved in 1377 iterations and 0.03 seconds
Optimal objective  3.221506012e+06
Minimum cost = $ 3221506.0117758485


Now that our variables are in 2 dimensions, double checking them can be a bit impractical; we will have 300 generator variables and over 1000 arc variables! lets see how the generator variables look.

In [32]:
for n in N:
    for t in T:
        if X[n,t].x >0:
            print('node',n, ': period',t, ': ',round(X[n,t].x,2))

node 20 : period 0 :  374.88
node 20 : period 1 :  400.0
node 20 : period 2 :  400.0
node 20 : period 3 :  400.0
node 20 : period 4 :  400.0
node 20 : period 5 :  400.0
node 35 : period 0 :  139.18
node 35 : period 1 :  300.2
node 35 : period 2 :  646.31
node 35 : period 3 :  818.0
node 35 : period 4 :  818.0
node 35 : period 5 :  789.31
node 37 : period 2 :  23.34
node 37 : period 3 :  123.57
node 37 : period 4 :  492.23
node 37 : period 5 :  73.92
node 45 : period 0 :  567.58
node 45 : period 1 :  833.62
node 45 : period 2 :  838.0
node 45 : period 3 :  838.0
node 45 : period 4 :  838.0
node 45 : period 5 :  838.0


Next email:

***

_Thank you for your help in optimising the use of our generators and network. This has enabled us to receive a capital budget for improving our production and delivery of electricity in the region._

_Firstly, we have funds to increase the capacity of three of our transmission lines by 50 MW (to 150 MW). Which lines we should increase?_

_Please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our generators._

***

This request is asking us to select the best arcs to 'switch on' to a higher capacity in order to optimise the network. This kind of problem  requires us to start looking at Mixed Integer Linear programming (MILP). Although it's called MILP, you will find that most of these problems actually use binary variables rather than interger variables. 

First thing we need to do is load the new data and start a new model. It is important to note that rather than using 150MW as a data point, we are going to use 50MW and add it to the existing transmission allowance. 

In [33]:
# Email 5 data

Num_Inc_Lines = 3 # Number of lines with extra transmission allowance
Extra_Arc_Cap = 50 # Extra transmission allowance(MW)

Our objective remains unchanged as we are still optimising energy transmission over the whole day. 

Binary variables are declared in the same way as regular continuous variables, but changing the vtype argument.

NB: vtype = CONTINUOUS is the default when the addVar argument is blank.

In [34]:
# Email 5 variables

# X gives amount generated by generators at node n in time period t 
X = {(n,t): m.addVar() for n in N for t in T}

# Y gives flow on arc a in time period t
Y = {(a,t): m.addVar() for a in A for t in T}

# I indicates whether the capacity is increased for all arcs
I = {a: m.addVar(vtype=GRB.BINARY) for a in A}

In [35]:
# Email 5 objective

m.setObjective(4*(quicksum(costs[n]*X[n,t] for n in costs for t in T)), GRB.MINIMIZE)

In this model, Gurobi will try to increase the capacity of a series of lines in the network. We need to tell the model to do two things; 1) To restrict the total number of increased transmission lines to 3, and 2) add 50MW capacity to the chosen transmission lines. 

The way gurobi achieves this is with binary variables. We have declared `I[a]` as the binary variable, so the only values `I[a]` can take on are 0 or 1.  

In [36]:
# restrict the number of extra-capacity transmission lines
m.addConstr(quicksum(I[a] for a in A) == Num_Inc_Lines)

<gurobi.Constr *Awaiting Model Update*>

This constraint is telling us that over every arc in the series of arcs, the sum of `I` must equal 3. Since they are binary, you basically get three `I[a]`'s that equal 1, and the rest will equal zero. This is important for our next constraint, because when `I[a] = 1`, that means we can apply any kind of data we want, while those that equal zero will be unaffected.

In [37]:
for t in T:            
    for a in A:
        if not a in highs:
            # Restrict to max capacity of lines + additional capacity if applicable
            m.addConstr(Y[a,t] <= lowlimit + Extra_Arc_Cap*I[a])

Now when Gurobi sets the `I = 1`, that transmission capacity will be 100 + 50, while the remaining lines will stay at 100. 

The rest of the constraints are as before, since we have not changed any of that data 

In [38]:
# Email 5 constraints

for t in T:    
    for n in N:
        # Balancing the power flow at each node and in each time period
        m.addConstr(quicksum(Y[a,t]*(1-loss*distance[a]) for a in A if arcs['Node2'][a] == n) + X[n,t]  ==
                    quicksum(Y[a,t] for a in A if arcs['Node1'][a] == n) + D[n][t])
        
        if n in supply:
            # Power generated by existing generators under capacity
            m.addConstr(X[n,t] <= supply[n])
        else: 
            # Forcing constraint
            m.addConstr(X[n,t] == 0)

m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3694 rows, 3660 columns and 10378 nonzeros
Model fingerprint: 0x2bfca191
Variable types: 3466 continuous, 194 integer (194 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+03]
Presolve removed 2337 rows and 2296 columns
Presolve time: 0.02s
Presolved: 1357 rows, 1364 columns, 4640 nonzeros
Variable types: 1188 continuous, 176 integer (176 binary)

Root relaxation: objective 3.213872e+06, 898 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3213871.75    0    5          - 3213871.75      -     -    0s
H    0     0                    3216986.7025 3213871.75  0.10%     -    0s
H    0    

We can take a look at the binary variable and see which arcs had their capacity increased. note the use of `> 0.9` rather than `== 1`, this kind of expression avoids issues associated with float data types.

In [39]:
for a in A:
    for t in T:
        if I[a].x> 0.9:
            print('arc',a, ': period',t, ': ',round(Y[a,t].x,2))

arc 53 : period 0 :  0.0
arc 53 : period 1 :  0.0
arc 53 : period 2 :  0.0
arc 53 : period 3 :  48.16
arc 53 : period 4 :  150.0
arc 53 : period 5 :  8.35
arc 95 : period 0 :  73.36
arc 95 : period 1 :  124.37
arc 95 : period 2 :  133.24
arc 95 : period 3 :  150.0
arc 95 : period 4 :  88.18
arc 95 : period 5 :  150.0
arc 177 : period 0 :  150.0
arc 177 : period 1 :  114.29
arc 177 : period 2 :  129.41
arc 177 : period 3 :  150.0
arc 177 : period 4 :  121.1
arc 177 : period 5 :  127.65


So we can see that arcs 85, 95 and 177 are selected as the extra capacity transmission lines, and for most periods of the day the capacity is above the 100 limit (although not always at the maxed 150) 

Using binary data as on/off switches is an immensely powerful technique for modelling. I have a feeling that future communications from Sparkland may require the use of this technique! 

***

_We also have funds to build a small gas generator at one of the existing nodes on the network, where there is not already a generator. This generator can supply up to 200 MW at a cost of \$78/MWh. Where should we build this generator? Note that any existing demand at the node where it is located will still need to be met._

_Please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our generators. We realise that this may affect your proposal on which transmission lines to upgrade._
***

Using our new technique, we should be able to declare a new binary variable that iterates over the non-generator nodes, restrict how many nodes can be turned on, and then model the additional cost associated with the new generators. Lets start with loading our new data, and creating our variables

In [40]:
# Email 6 data

Num_New_Generator = 1 # Number of new generators to be constructed
New_Capacity = 200 # Capacity of the new generator
New_Cost = 78 # Cost of generation for the new generator ($/MWh)

In [41]:
# Email 6 variables

# X gives amount of energy generated by generators at node n in time period t 
X = {(n,t): m.addVar() for n in N for t in T}

# Y gives flow of energy on arc a in time period t
Y = {(a,t): m.addVar() for a in A for t in T}

# I indicates whether the capacity is increased for all arcs
I = {a: m.addVar(vtype=GRB.BINARY) for a in A}

# N_G indicates whether the node is chosen as a new generator
N_G = {n: m.addVar(vtype=GRB.BINARY) for n in N if n not in supply} 

We now have a new generator that generates power and costs money. Since we are trying to minimize cost, this will require an alteration to our objective:

In [42]:
# Email 6 objectives 

m.setObjective(4*(quicksum(costs[n]*X[n,t] for n in costs for t in T)
               + quicksum(New_Cost*X[n,t] for n in N for t in T if n not in costs)), GRB.MINIMIZE)

This additional line will pick up the energy generated by the new generator and multiply it by the new generator cost.

Like before, we need a constraint that limits the number of nodes that receive a new generator, restricting it to nodes that do not already have a generator. 

Most of the other constraints remain the same, however we do augment one constraint in a very subtle but important way

In [43]:
# Email 6 constraints

# restrict the number of extra-capacity transmission lines
m.addConstr(quicksum(I[a] for a in A) == Num_Inc_Lines)

# restrict the number of new gas generator(s)
m.addConstr(quicksum(N_G[n] for n in N if n not in supply) == Num_New_Generator)

for t in T:    
    for a in A:
        if not a in highs:
            # Restrict to max capacity of lines + additional capacity if applicable
            m.addConstr(Y[a,t] <= lowlimit + Extra_Arc_Cap*I[a])
    
    for n in N:
        # Balancing the power flow at each node and in each time period
        m.addConstr(quicksum(Y[a,t]*(1-loss*distance[a]) for a in A if arcs['Node2'][a] == n) + X[n,t]  ==
                    quicksum(Y[a,t] for a in A if arcs['Node1'][a] == n) + D[n][t])
        
        if n in supply:
            # Power generated by existing generators under capacity
            m.addConstr(X[n,t] <= supply[n])
        else: 
            # Forcing constraint + Capacity of the gas generator if applicable at node n
            m.addConstr(X[n,t] <= N_G[n]*New_Capacity)

Our forcing constraint has taken on a very new look, but it still largely performs the same role as before. Here is how to interpret it:
* when `N_G[n] == 0`, it is a normal node, so `X[n,t] <= 0 * New_Capacity == 0`. This is basically the forcing constraint as before 
* when `N_G[n] == 1`, it is a new generator, so `X[n,t] <= New_Capacity`, which we can now use in our objective


In [44]:
m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5352 rows, 5364 columns and 15934 nonzeros
Model fingerprint: 0x8df14459
Variable types: 4930 continuous, 434 integer (434 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [3e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]

MIP start from previous solve produced solution with objective 3.20687e+06 (0.07s)
MIP start from previous solve produced solution with objective 3.20147e+06 (0.07s)
MIP start from previous solve produced solution with objective 3.20032e+06 (0.18s)
Loaded MIP start from previous solve with objective 3.20032e+06

Presolve removed 3718 rows and 3678 columns
Presolve time: 0.04s
Presolved: 1634 rows, 1686 columns, 5514 nonzeros
Variable types: 1464 continuous, 222 integer (222 binary)

Root relaxation: objective 3.194360e+06, 1494 iterations, 0.03 seco

Once again, we can check which node was switched on, and we can also see what the generation profile looks like. 

In [45]:
for t in T:
    for n in N: 
        if n not in supply:
            if N_G[n].x > 0.9:
                print(n, N_G[n].x, round(X[n,t].x,2))

33 1.0 0.0
33 1.0 0.0
33 1.0 133.13
33 1.0 200.0
33 1.0 200.0
33 1.0 200.0


No demand overnight, maxed out demand in peak periods. Makes perfect sense. Okay on to the next email:
***
_Unfortunately the local government has declined our application to build the new generator at Node 33. Could you propose an alternative site we should use? We realise that this may affect your earlier proposal on which transmission lines to upgrade._

_Please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our generators._
***

Okay so our previous analysis found the optimal configuration for a new generator. Our new model is going to only include one extra constraint which specifically excludes node 33

In [46]:
# Email 7 data

Declined_node = 33

In [47]:
# Email 7 constraint

m.addConstr(N_G[Declined_node] == 0)

<gurobi.Constr *Awaiting Model Update*>

In this case, we do not need to redeclare our variables, objective or constraints, so lets just go.

In [48]:
m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5353 rows, 5364 columns and 15935 nonzeros
Model fingerprint: 0x3f6261c8
Variable types: 4930 continuous, 434 integer (434 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [3e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R5352 by 1.000000000

Presolve removed 3725 rows and 3685 columns
Presolve time: 0.05s
Presolved: 1628 rows, 1679 columns, 5495 nonzeros
Variable types: 1458 continuous, 221 integer (221 binary)

Root relaxation: objective 3.194726e+06, 1408 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 31

In [49]:
for t in T:
    for n in N: 
        if n not in supply:
            if N_G[n].x > 0.9:
                print(n, N_G[n].x, round(X[n,t].x,2))

26 1.0 0.0
26 1.0 0.0
26 1.0 126.21
26 1.0 200.0
26 1.0 200.0
26 1.0 200.0


Excluding node 33 causes the model to optimize for node 26 instead, and it's under $100 more expensive, so maybe the local council has made the correct choice. 

***

_We're excited to let you know that we have the go-ahead to build the gas generator. Due to the design of the gas generator, we can only run it during four time periods (16 hours) each day. Given our current demands, which time periods should it be operated? We realise that this may affect your earlier proposals on which transmission lines to upgrade and even where we should build the new gas generator._

_Please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our generators._

***

This request seems a little redundant; we have already analysed the result of the last modelling exercise and two of the 6 periods are already at zero. We will implement the new requirements as requested, but the prediction is that the objective value will be the same. However they raise a good point, does the placement of this new generator alter the high capacity lines we optimised earlier?

Lets start with the new data (max number of periods)

In [50]:
# Email 8 data

New_Periods = 4 # Number of periods that the gas generator can run per day

We'll need a new binary variable to select the optimal periods

In [51]:
# Email 8 variable

#X gives amount generated by generators (including the gas generator) at node n in time period t
X = {(n,t): m.addVar() for n in N for t in T}

# Y gives flow on arc a in time period t
Y = {(a,t): m.addVar() for a in A for t in T}

# I indicates whether the capacity is increased for all arcs
I = {a: m.addVar(vtype=GRB.BINARY) for a in A}

# N_G indicates whether the node is chosen as a new generator
N_G = {n: m.addVar(vtype=GRB.BINARY) for n in N if n not in supply} 

# P indicates in which periods the gas generator should run
P = {t: m.addVar(vtype=GRB.BINARY) for t in T}


In [52]:
# Email 8 objective

m.setObjective(4*(quicksum(costs[n]*X[n,t] for n in costs for t in T)
               + quicksum(New_Cost*X[n,t] for n in N for t in T if n not in costs)), GRB.MINIMIZE)

New constraints too, we'll add them to the previous ones. 

In [53]:
# Email 8 constraints

# restrict the number of extra-capacity transmission lines
m.addConstr(quicksum(I[a] for a in A) == Num_Inc_Lines)

# Total number of time periods where the gas generator is operational
m.addConstr(quicksum(P[t] for t in T) == New_Periods)

# Exclude building new generator at the declined node
m.addConstr(N_G[Declined_node] == 0)

# Total number of new gas generator(s)
m.addConstr(quicksum(N_G[n] for n in N if n not in supply) == Num_New_Generator)

for t in T:    
    for a in A:
        if not a in highs:
            # Restrict to max capacity of lines + additional capacity if applicable
            m.addConstr(Y[a,t] <= lowlimit + Extra_Arc_Cap*I[a])
    
    for n in N:
        # Balancing the power flow at each node and in each time period
        m.addConstr(quicksum(Y[a,t]*(1-loss*distance[a]) for a in A if arcs['Node2'][a] == n) + X[n,t]  ==
                    quicksum(Y[a,t] for a in A if arcs['Node1'][a] == n) + D[n][t])
        
        if n in supply:
            # Power generated by existing generators under capacity
            m.addConstr(X[n,t] <= supply[n])
        else: 
            # Forcing constraint + Capacity of the gas generator at node n
            m.addConstr(X[n,t] <= N_G[n] * New_Capacity)
            # Capacity of the gas generator in time period t
            m.addConstr(X[n,t] <= P[t] * New_Capacity)

In [54]:
m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7289 rows, 7074 columns and 22050 nonzeros
Model fingerprint: 0xd1a171a7
Variable types: 6394 continuous, 680 integer (680 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [3e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]

MIP start from previous solve produced solution with objective 3.21465e+06 (0.08s)
MIP start from previous solve produced solution with objective 3.20147e+06 (0.08s)
MIP start from previous solve produced solution with objective 3.20041e+06 (0.19s)
Loaded MIP start from previous solve with objective 3.20041e+06

Presolve removed 5390 rows and 5389 columns
Presolve time: 0.08s
Presolved: 1899 rows, 1685 columns, 6041 nonzeros
Variable types: 1458 continuous, 227 integer (227 binary)

Root relaxation: objective 3.194726e+06, 1424 iterations, 0.03 seco

In [55]:
for t in T:
    for n in N: 
        if n not in supply:
            if N_G[n].x > 0.9:
                print('node',n, ': period',t, ': ',round(X[n,t].x,2))

node 26 : period 0 :  0.0
node 26 : period 1 :  0.0
node 26 : period 2 :  126.21
node 26 : period 3 :  200.0
node 26 : period 4 :  200.0
node 26 : period 5 :  200.0


Well it doesn't tell us much as it looks the same as before, but let's make sure the binary variable is working as expected

In [56]:
for t in T:
    print('period' , t, ': ' ,P[t].x)

period 0 :  -0.0
period 1 :  0.0
period 2 :  1.0
period 3 :  1.0
period 4 :  1.0
period 5 :  1.0


Very good, periods zero and one are switched off. And what about our high capacity arcs? 

In [57]:
for a in A: 
    for t in T:
        if I[a].x > 0.9:
            print('arc',a, ': period',t, ': ',round(Y[a,t].x,2))

arc 85 : period 0 :  112.22
arc 85 : period 1 :  145.67
arc 85 : period 2 :  150.0
arc 85 : period 3 :  150.0
arc 85 : period 4 :  150.0
arc 85 : period 5 :  150.0
arc 95 : period 0 :  73.36
arc 95 : period 1 :  124.37
arc 95 : period 2 :  133.24
arc 95 : period 3 :  150.0
arc 95 : period 4 :  149.4
arc 95 : period 5 :  150.0
arc 177 : period 0 :  150.0
arc 177 : period 1 :  114.29
arc 177 : period 2 :  129.41
arc 177 : period 3 :  150.0
arc 177 : period 4 :  150.0
arc 177 : period 5 :  127.65


Appears to be exactly the same high capacity arcs and the same transmission profile throughout the day. Okay, next email:

***
_In addition to the gas generator, we can also build a solar farm at one of the nodes. This will produce electricity over the day as follows:_

|Time Period|0–4|4–8|8–12|12–16|16–20|20–24|
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | 
|Supply (MW)|0|20|120|110|20|0|

_The cost of the solar electricity is $42/MWh. Where should we build this solar farm? Note that any existing demand at the node where it is located will still need to be met._

_Please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our combined generators. We realise that this may affect your earlier proposals on which transmission lines to upgrade and where we should build the new gas generator._

***

Finally, some solar generation! I was becoming conflicted helping this fossil energy generator maximise their shareholders' profits. This will be very similar to our new gas generator process (new objective, variable, and constraints) with a bit more data surrounding generation profile too


In [58]:
# Email 9 data

Num_Solar_Farm = 1 # Number of solar farm(s)
Supply_SF = [0,20,120,110,20,0] # Capacity of the solar farm in different time periods 
Cost_SF = 42 # Cost of energy production at the solar farm p/MWh

In [59]:
# Email 9 variables

# X gives amount generated by generators (including the gas generator) at node n in time period t 
X = {(n,t): m.addVar() for n in N for t in T}

# Y gives flow on arc a in time period t
Y = {(a,t): m.addVar() for a in A for t in T}

# I indicates whether the capacity is increased for arc a 
I = {a: m.addVar(vtype=GRB.BINARY) for a in A}

# N_G indicates whether the node is chosen as a new generator
N_G = {n: m.addVar(vtype=GRB.BINARY) for n in N if n not in supply} 

# P indicates in which periods the gas generator should run
P = {t: m.addVar(vtype=GRB.BINARY) for t in T}

# S indicates whether the node is chosen to build a solar farm
S = {n: m.addVar(vtype=GRB.BINARY) for n in N}

# Z gives amount generated by the solar farm at node n in time period t
Z = {(n,t): m.addVar() for n in N for t in T}

In [60]:
# Email 9 objective

m.setObjective(4*(quicksum(costs[n]*X[n,t] for n in costs for t in T)
                + quicksum(New_Cost*X[n,t] for n in N for t in T if n not in costs)
                + quicksum(Cost_SF*Z[n,t] for n in N for t in T)), GRB.MINIMIZE)

In [61]:
# Email 9 constraints 

# restrict the number of extra-capacity transmission lines
m.addConstr(quicksum(I[a] for a in A) == Num_Inc_Lines)

# Total number of time periods where the gas generator is operational
m.addConstr(quicksum(P[t] for t in T) == New_Periods)

# Exclude building new generator at the declined node
m.addConstr(N_G[Declined_node] == 0)

# Total number of new gas generator(s)
m.addConstr(quicksum(N_G[n] for n in N if n not in supply) == Num_New_Generator)

# Total number of solar farm(s)
m.addConstr(quicksum(S[n] for n in N) == Num_Solar_Farm)

for t in T:
    for a in A:
        if not a in highs:
            # Capacity of lines with additional capacity if applicable
            m.addConstr(Y[a,t] <= lowlimit + Extra_Arc_Cap*I[a])
            
    for n in N:
        # Balancing the power flow at each node and in each time period
        m.addConstr(quicksum(Y[a,t]*(1-loss*distance[a]) for a in A if arcs['Node2'][a] == n) + X[n,t] + Z[n,t]  ==
                    quicksum(Y[a,t] for a in A if arcs['Node1'][a] == n) + D[n][t])
        
        # Power generated by the solar farm at node n and in time period t
        m.addConstr(Z[n,t] <= S[n]*Supply_SF[t])
        
        if n in supply:
            # Power generated by existing generators under capacity
            m.addConstr(X[n,t] <= supply[n])
            
        else: 
            # Capacity of new gas generator at node n
            m.addConstr(X[n,t] <= N_G[n]*New_Capacity)
            
            # Capacity of new gas generator in time period t
            m.addConstr(X[n,t] <= P[t] * New_Capacity)
            

Similar to the new gas generator, we have constraint that limits the number of solar generators we can construct by allowing variable `Z[n,t] == 1`  once. We then use that variable to 'switch on' the node and allow it to generate power as per the generation profile data.

In [62]:
m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9526 rows, 9134 columns and 29015 nonzeros
Model fingerprint: 0xc2e4c72b
Variable types: 8158 continuous, 976 integer (976 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]

MIP start from previous solve produced solution with objective 3.17408e+06 (0.10s)
MIP start from previous solve produced solution with objective 3.16223e+06 (0.11s)
MIP start from previous solve produced solution with objective 3.15875e+06 (0.21s)
Loaded MIP start from previous solve with objective 3.15875e+06

Presolve removed 7426 rows and 7199 columns
Presolve time: 0.13s
Presolved: 2100 rows, 1935 columns, 6691 nonzeros
Variable types: 1658 continuous, 277 integer (277 binary)

Root relaxation: objective 3.152702e+06, 1983 iterations, 0.04 seco

So where did this new Solar farm get constructed, and how muchpower is it spitting out?

In [63]:
for n in N:
    for t in T:
        if S[n].x > 0.9:
            print('solar node',n, ': period',t, ':',round(Z[n,t].x,2))

solar node 40 : period 0 : 0.0
solar node 40 : period 1 : 20.0
solar node 40 : period 2 : 120.0
solar node 40 : period 3 : 110.0
solar node 40 : period 4 : 20.0
solar node 40 : period 5 : 0.0


Constructed at Node 40 and the farm is maxed out. This makes sense as this is by far the cheapest source of power in the grid, even if you account for losses from a distant node. Let's make sure our high capacity arcs are still in the same place


In [64]:
for a in A:
    if I[a].x > 0.9:
        print('high arc:',a)

high arc: 85
high arc: 95
high arc: 177


Okay nothing has changed here, so on to the next email

***
_The costs we have given you for our original four generators are rather simplistic. In practice, the generators operate efficiently up to 60% of their capacity but after that they become more expensive to run. We estimate this increase in cost to be 30% on top of the original values we gave you._

_For example, the generator at Node 20 will run at a cost of \\$65/MWh when supplying no more than 243.6 MW. Beyond that threshold, it will then cost \\$84.5/MWh to run._

_Taking these revised costs into account, please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our combined generators. We realise that this may affect your earlier proposals on which transmission lines to upgrade and where we should build the new gas generator and solar farm._
***

Okay now things are starting to look more like a real network, where costs are not static. Binary variables can be used to switch between higher and lower costs as well as just on/off reasons. The data for this is actually very straightforward: a simple threshold and simple percentage increase:

In [65]:
# Email 10 data

Cap_Threshold = 0.6 # Threshold of generators above which power production is less efficient
Extra_Cost = 0.3 # Cost increase (%) of power when generator production exceeds 

We will need to include two new variables. One of them `'O'` will be an indicator that switches on when the generator is operating above the threshold. Another continuous variable `'E'` is required to track the amount energy being produced over the threshold, as this energy will attract a different cost. As a result, our existing generator variable Y will now only track energy produced under the threshold.

In [66]:
# Email 10 variables

# X gives flow on arc a in time period t
X = {(n,t): m.addVar() for n in N for t in T}

# Y gives amount generated by generators (including the gas generator) at node n in time period t within the efficient threshold
Y = {(a,t): m.addVar() for a in A for t in T}

# I indicates whether the capacity is increased for arc a
I = {a: m.addVar(vtype=GRB.BINARY) for a in A}

# N_G indicates whether the node is chosen as a new generator
N_G = {n: m.addVar(vtype=GRB.BINARY) for n in N if n not in supply} 

# P indicates in which periods the gas generator should run
P = {t: m.addVar(vtype=GRB.BINARY) for t in T}

# S indicates whether the node is chosen to build a solar farm
S = {n: m.addVar(vtype=GRB.BINARY) for n in N}

# Z gives amount generated by the solar farm at node n in time period t
Z = {(n,t): m.addVar() for n in N for t in T}

# O indicates whether generator n is operating over 60% of its capacity in time period t
O = {(n,t): m.addVar(vtype=GRB.BINARY) for n in N for t in T}

# E gives the extra amount generated by generator n over 60% of their capacities in time period t
E = {(n,t): m.addVar() for n in N for t in T}

New objective is required here as we are producing energy that has different costs associated with it. 

In [67]:
# Email 10 objective 

m.setObjective(4*(quicksum(costs[n]*X[n,t] for n in costs for t in T)
                + quicksum(New_Cost*X[n,t] for n in N for t in T if n not in costs)
                + quicksum(Cost_SF*Z[n,t] for n in N for t in T)
                + quicksum(costs[n]*(1+Extra_Cost)*E[n,t] for n in costs for t in T)), GRB.MINIMIZE)

Quite a few constraints to introduce this time:
* The new energy variable `'E'` must be included in the balancing constraint
* Both of the threshold constraints do not apply to non-generating nodes
* The existing supply constraint needs to be adjusted to lower the generator capacity to the threshold value
* New constraint to deal with the energy produced above the threshold

By enforcing a higher price for power generated above the threshold, our model automatically optimises the system and will try to avoid producing energy at an expensive rate unless there is no other choice. 

In [68]:
# Email 10 constraints 

# restrict the number of extra-capacity transmission lines
m.addConstr(quicksum(I[a] for a in A) == Num_Inc_Lines)

# Total number of time periods where the gas generator is operational
m.addConstr(quicksum(P[t] for t in T) == New_Periods)

# Exclude building new generator at the declined node
m.addConstr(N_G[Declined_node] == 0)

# Total number of new gas generator(s)
m.addConstr(quicksum(N_G[n] for n in N if n not in supply) == Num_New_Generator)

# Total number of solar farm(s)
m.addConstr(quicksum(S[n] for n in N) == Num_Solar_Farm)

for t in T:
    for a in A:
        if not a in highs:
            # Capacity of lines with additional capacity if applicable
            m.addConstr(Y[a,t] <= lowlimit + Extra_Arc_Cap*I[a])

    for n in N:
        # Balancing the power flow at each node and in each time period
        m.addConstr(quicksum(Y[a,t]*(1-loss*distance[a]) for a in A if arcs['Node2'][a] == n) + X[n,t] + Z[n,t]  + E[n,t] ==
                    quicksum(Y[a,t] for a in A if arcs['Node1'][a] == n) + D[n][t])
        
        # Power generated by the solar farm at node n and in time period t
        m.addConstr(Z[n,t] <= S[n]*Supply_SF[t])
        
        if n in supply:
            # Power generated by existing generators under threshold
            m.addConstr(X[n,t] <= supply[n]*Cap_Threshold)
            
            # Power generated by existing generators beyond the threshold
            m.addConstr(E[n,t] <= O[n,t]*supply[n]*(1-Cap_Threshold))
            
        else: 
            # Capacity of new gas generator at node n
            m.addConstr(X[n,t] <= N_G[n]*New_Capacity)
            
            # Capacity of new gas generator in time period t
            m.addConstr(X[n,t] <= P[t] * New_Capacity)
                        
            # Threshold is not applicable to non-generator nodes
            m.addConstr(O[n,t] == 0)
            m.addConstr(E[n,t] == 0)
            

That should hopefully cover it

In [69]:
m.optimize()
print("Minimum cost = $",m.objVal)


Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12339 rows, 11794 columns and 36880 nonzeros
Model fingerprint: 0xafaac029
Variable types: 10222 continuous, 1572 integer (1572 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [2e+02, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]

MIP start from previous solve produced solution with objective 3.35824e+06 (0.13s)
MIP start from previous solve produced solution with objective 3.3215e+06 (0.13s)
MIP start from previous solve produced solution with objective 3.31917e+06 (0.23s)
MIP start from previous solve produced solution with objective 3.31681e+06 (0.28s)
MIP start from previous solve produced solution with objective 3.31653e+06 (0.62s)
Loaded MIP start from previous solve with objective 3.31653e+06

Presolve removed 10239 rows and 9835 columns
Presolve time: 0.18s
Preso

Let's do our _post hoc_ analysis and see what has happened

In [70]:
for n in N:
    for t in T:
        if n in supply:
            print('node',n,': period',t,': under threshold =', round(X[n,t].x,2), ': over threshold =',round(E[n,t].x,2))
        elif N_G[n].x >0.9:
            print('new node',n,': period',t, ':', round(X[n,t].x,2))
        elif S[n].x >0.9:
            print('solar node',n,': period',t, ':', round(Z[n,t].x,2))
            
for a in A: 
    for t in T:
        if I[a].x > 0.9:
            print('arc',a, ': period',t, ': ',round(Y[a,t].x,2))
            

node 20 : period 0 : under threshold = 243.6 : over threshold = 0.0
node 20 : period 1 : under threshold = 243.6 : over threshold = 0.0
node 20 : period 2 : under threshold = 243.6 : over threshold = 12.6
node 20 : period 3 : under threshold = 243.6 : over threshold = 162.4
node 20 : period 4 : under threshold = 243.6 : over threshold = 162.4
node 20 : period 5 : under threshold = 243.6 : over threshold = 162.4
new node 26 : period 0 : 0.0
new node 26 : period 1 : 0.0
new node 26 : period 2 : 200.0
new node 26 : period 3 : 200.0
new node 26 : period 4 : 200.0
new node 26 : period 5 : 200.0
node 35 : period 0 : under threshold = 329.46 : over threshold = 0.0
node 35 : period 1 : under threshold = 490.8 : over threshold = 0.0
node 35 : period 2 : under threshold = 490.8 : over threshold = 0.0
node 35 : period 3 : under threshold = 490.8 : over threshold = 0.0
node 35 : period 4 : under threshold = 490.8 : over threshold = 176.83
node 35 : period 5 : under threshold = 490.8 : over thresho

So what have we discovered here? a few insights:
* Our solar node has moved from node 40 to node 41
* we have a different set of high capacity arcs
* period 4 (between 4-8pm) is a huge peak time, where generators go over the threshold significantly
* despite this, our most expensive generator (37) never goes over the threshold

Could these insights allow Sparkland to introduce alternative (cheaper) energy infrastructure to service the evening peak? It's a fraught problem; dealing with the evening peak while solar generation tapers off is a live issue that networks are grappling with.

Anyway, we come to our final email, which means we will have the final model ready to go. 

***

_To counter the increased costs of peak production, we have been working with customers on a scheme where they will commit to reduce their demand by 10% for one time period each day. We can specify the time periods when this will happen for each node but no more than 9 nodes can be reduced during the same time period._

_Please provide us with the optimal total cost over the day for meeting the demand in each of the six time periods from our combined generators. We again realise that this may affect your earlier proposals on which transmission lines to upgrade and where we should build the new gas generator and solar farm._

_We look forward to reading your final report._

***

One final tweak to make the optimal system. Sometimes reading these kinds of verbal/written instructions can be a bit confusing, and it can certainly help to write them down as data to help refine thinking.

In [71]:
# Final data
Reduced_demand = 0.1 # Percentage by which the demand is reduced
Num_Reduce_Period = 1 # Number of periods that a node can reduce its demand in a day
Num_Reduce_Node = 9 # Number of nodes that can reduce their demand in a single time period

We need a new binary variable to switch on the lower-demand node

In [72]:
# Final Variables

# X gives flow on arc a in time period t
X = {(n,t): m.addVar() for n in N for t in T}

# Y gives amount generated by generators (including the gas generator) at node n in time period t within the efficient threshold
Y = {(a,t): m.addVar() for a in A for t in T}

# I indicates whether the capacity is increased for arc a
I = {a: m.addVar(vtype=GRB.BINARY) for a in A}

# N_G indicates whether the node is chosen as a new generator
N_G = {n: m.addVar(vtype=GRB.BINARY) for n in N if n not in supply} 

# P indicates in which periods the gas generator should run
P = {t: m.addVar(vtype=GRB.BINARY) for t in T}

# S indicates whether the node is chosen to build a solar farm
S = {n: m.addVar(vtype=GRB.BINARY) for n in N}

# Z gives amount generated by the solar farm at node n in time period t
Z = {(n,t): m.addVar() for n in N for t in T}

# O indicates whether generator n is operating over 60% of its capacity in time period t
O = {(n,t): m.addVar(vtype=GRB.BINARY) for n in N for t in T}

# E gives the extra amount generated by generator n over 60% of their capacities in time period t
E = {(n,t): m.addVar() for n in N for t in T}

# R indicates whether a node reduces its demand in a time period.
R = {(n,t): m.addVar(vtype=GRB.BINARY) for n in N for t in T}

Objective remains the same as we are not introducing any new generator costs.

In [73]:
# Final Objective

m.setObjective(4*(quicksum(costs[n]*X[n,t] for n in costs for t in T)
                + quicksum(New_Cost*X[n,t] for n in N for t in T if n not in costs)
                + quicksum(Cost_SF*Z[n,t] for n in N for t in T)
                + quicksum(costs[n]*(1+Extra_Cost)*E[n,t] for n in costs for t in T)), GRB.MINIMIZE)

In [74]:
# Final Constraints

for n in N:
    # Number of time periods for which a node should reduce demand per day
    m.addConstr(quicksum(R[n,t] for t in T) == Num_Reduce_Period)

# restrict the number of extra-capacity transmission lines
m.addConstr(quicksum(I[a] for a in A) == Num_Inc_Lines)

# Total number of time periods where the gas generator is operational
m.addConstr(quicksum(P[t] for t in T) == New_Periods)

# Exclude building new generator at the declined node
m.addConstr(N_G[Declined_node] == 0)

# Total number of new gas generator(s)
m.addConstr(quicksum(N_G[n] for n in N if n not in supply) == Num_New_Generator)

# Total number of solar farm(s)
m.addConstr(quicksum(S[n] for n in N) == Num_Solar_Farm)

for t in T:
    
    # Maximum number of nodes to reduce demand in one time period 
    m.addConstr(quicksum(R[n,t] for n in N) <= Num_Reduce_Node)
    
    for a in A:
        if not a in highs:
            # Capacity of lines with additional capacity if applicable
            m.addConstr(Y[a,t] <= lowlimit + Extra_Arc_Cap*I[a])

    for n in N:
        # Balancing the power flow at each node and in each time period
        m.addConstr(quicksum(Y[a,t]*(1-loss*distance[a]) for a in A if arcs['Node2'][a] == n) + X[n,t] + Z[n,t]  + E[n,t] ==
                    quicksum(Y[a,t] for a in A if arcs['Node1'][a] == n) + D[n][t]*(1-R[n,t]*Reduced_demand))
        
        # Power generated by the solar farm at node n and in time period t
        m.addConstr(Z[n,t] <= S[n]*Supply_SF[t])
        
        if n in supply:
            # Power generated by existing generators under threshold
            m.addConstr(X[n,t] <= supply[n]*Cap_Threshold)
            
            # Power generated by existing generators beyond the threshold
            m.addConstr(E[n,t] <= O[n,t]*supply[n]*(1-Cap_Threshold))
            
        else: 
            # Capacity of new gas generator at node n
            m.addConstr(X[n,t] <= N_G[n]*New_Capacity)
            
            # Capacity of new gas generator in time period t
            m.addConstr(X[n,t] <= P[t] * New_Capacity)
                        
            # Threshold is not applicable to non-generator nodes
            m.addConstr(O[n,t] == 0)
            m.addConstr(E[n,t] == 0)

In [75]:
m.optimize()
print("Minimum cost = $",m.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15208 rows, 14754 columns and 45621 nonzeros
Model fingerprint: 0xf54eccd8
Variable types: 12286 continuous, 2468 integer (2468 binary)
Coefficient statistics:
  Matrix range     [8e-01, 3e+02]
  Objective range  [2e+02, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]

MIP start from previous solve produced solution with objective 3.27522e+06 (0.14s)
MIP start from previous solve produced solution with objective 3.24333e+06 (0.15s)
MIP start from previous solve produced solution with objective 3.23475e+06 (0.26s)
MIP start from previous solve produced solution with objective 3.23342e+06 (0.37s)
MIP start from previous solve produced solution with objective 3.23303e+06 (0.62s)
Loaded MIP start from previous solve with objective 3.23303e+06

Presolve removed 13052 rows and 12495 columns
Presolve time: 0.22s
Pre

Okay we have our final model and our most optimal price for running the network per day. Let's run a few numbers and see what our final configuration looks like.

In [76]:
print('Nodes that experienced reduced demand in each period:')
for t in T:
    for n in N:
        if R[n,t].x > 0.9:
            print('period',t,': node',n)

Nodes that experienced reduced demand in each period:
period 0 : node 12
period 0 : node 20
period 0 : node 35
period 0 : node 37
period 0 : node 45
period 1 : node 1
period 1 : node 5
period 1 : node 15
period 1 : node 27
period 1 : node 30
period 1 : node 34
period 1 : node 38
period 1 : node 42
period 1 : node 46
period 2 : node 7
period 2 : node 10
period 2 : node 16
period 2 : node 17
period 2 : node 28
period 2 : node 32
period 2 : node 39
period 2 : node 43
period 2 : node 47
period 3 : node 4
period 3 : node 6
period 3 : node 11
period 3 : node 19
period 3 : node 21
period 3 : node 22
period 3 : node 26
period 3 : node 29
period 3 : node 48
period 4 : node 3
period 4 : node 9
period 4 : node 14
period 4 : node 18
period 4 : node 23
period 4 : node 24
period 4 : node 25
period 4 : node 33
period 4 : node 41
period 5 : node 0
period 5 : node 2
period 5 : node 8
period 5 : node 13
period 5 : node 31
period 5 : node 36
period 5 : node 40
period 5 : node 44
period 5 : node 49


There are 6 periods during the day, with 9 demand-reduced nodes allowed per time period (54), but there are only 50 nodes in total. Since each node in only allowed to be reduced once per day, you will notice that time period 0 only has 5 nodes. This makes sense since time period 0 is in the middle of the night and has the lowest demand.

Let's double check our generator locations and extra-capacity arcs

In [77]:
for n in N:
    for t in T:
        if n in supply:
            print('node',n,': period',t,': under threshold =', round(X[n,t].x,2), ': over threshold =',round(E[n,t].x,2))
        elif N_G[n].x >0.9:
            print('new node',n,': period',t, ':', round(X[n,t].x,2))
        elif S[n].x >0.9:
            print('solar node',n,': period',t, ':', round(Z[n,t].x,2))
            
for a in A: 
    for t in T:
        if I[a].x > 0.9:
            print('arc',a, ': period',t, ': ',round(Y[a,t].x,2))

node 20 : period 0 : under threshold = 243.6 : over threshold = 0.0
node 20 : period 1 : under threshold = 243.6 : over threshold = 0.0
node 20 : period 2 : under threshold = 243.6 : over threshold = 0.0
node 20 : period 3 : under threshold = 243.6 : over threshold = 154.43
node 20 : period 4 : under threshold = 243.6 : over threshold = 162.4
node 20 : period 5 : under threshold = 243.6 : over threshold = 162.4
new node 26 : period 0 : 0.0
new node 26 : period 1 : 0.0
new node 26 : period 2 : 200.0
new node 26 : period 3 : 200.0
new node 26 : period 4 : 200.0
new node 26 : period 5 : 200.0
node 35 : period 0 : under threshold = 328.04 : over threshold = 0.0
node 35 : period 1 : under threshold = 490.8 : over threshold = 0.0
node 35 : period 2 : under threshold = 490.8 : over threshold = 0.0
node 35 : period 3 : under threshold = 490.8 : over threshold = 0.0
node 35 : period 4 : under threshold = 490.8 : over threshold = 94.33
node 35 : period 5 : under threshold = 490.8 : over threshol

Our generator locations all remain the same, but high capacity arc line 85 is replaced with arc 188.

****

And that's it for Sparkland's LP MILP optimisation process. I have put the code in a single .py file so you don't have to keep hunting through this tutorial. 

As mentioned earlier this is not simply an academic exercise: the Australian Energy Market Operator (AEMO) use LP to determine the spot market price across the National Energy Market. The real network would incorporate a load of other constraints and would probably be dealing with some rubbery demand figures requiring some stochastic considerations... which is exacly the topic of our next tutorial!

Many thanks to my co-collaborators Ulysses Lam and Julie Redulla with all their help with this project. 

And massive shout out to Dr. Michael Bulmer for putting together the best and most engaging Operations Research course going around. 